In [1]:
import skimage.io as io
import time
import numpy as np
import cv2
import os

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from Features.preprocessing import *
from Features.LocalBinaryPattern import *
from Features.ChainCodesFeatures import *
from Features.PolygonFeatures import * 
from Features.WindowFeatures import *
from Features.windows import *

In [2]:
def Feature_Extractor(img):
    img = cutHandWriting(img)
    
    img = cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))

    
    #lbp = LocalBinaryPatterns(24,8)
    #lbp_features = lbp.describe(img)
    #st = time.time()
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #en = time.time()
#     print("Contours : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #windows, clusters, clusteredWindows = getClusteredWindows(img,13,10)
    #en = time.time()
#     print("Kesho : ",(en-st)*1000,"ms")
    
    #st = time.time()
    F5 = Chain_Codes_Feature_Extractor(img,contours,None)
    #en = time.time()
#     print("Ramzy : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #f10, f11, f12, f13, f14 = PolygonFeatures(contours)
    #en = time.time()
#     print("Ibrahim Lefta : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #Windows = list()
    
    #for i in range(clusteredWindows.shape[0]):
        #print(clusteredWindows[i])
        #Windows  += list(WindowFeatures(clusteredWindows[i].astype(np.uint8)))
    #en = time.time()
#     print("October : ",(en-st)*1000,"ms")
    return list(F5)
    #return F5
    Features = list(F1)+list(F2)+list(F3)+list(F4)+list(F5)+list(F7)+list(F8)+list(F9)+list(f10)+list(f11)+list(f12)+list(f13)+list(f14)+list(lbp_features)+Windows

In [3]:
correct = []
f = open("correct.txt", "r")
for x in f:
    correct.append(int(x)) 

In [4]:
test_cases = os.listdir("TestData")
test_cases.sort()

In [5]:
i = 0 
acc = 0
for case in test_cases:
    writer1_1  = cv2.imread('TestData/'+case+"/1/1.png", cv2.IMREAD_GRAYSCALE)
    writer1_2  = cv2.imread('TestData/'+case+"/1/2.png", cv2.IMREAD_GRAYSCALE)
    writer2_1  = cv2.imread('TestData/'+case+"/2/1.png", cv2.IMREAD_GRAYSCALE)
    writer2_2  = cv2.imread('TestData/'+case+"/2/2.png", cv2.IMREAD_GRAYSCALE)
    writer3_1  = cv2.imread('TestData/'+case+"/3/1.png", cv2.IMREAD_GRAYSCALE)
    writer3_2  = cv2.imread('TestData/'+case+"/3/2.png", cv2.IMREAD_GRAYSCALE)
    test_img =  cv2.imread('TestData/'+case+"/test.png", cv2.IMREAD_GRAYSCALE)

    st = time.time()
    '''
    f1_1_5 , f1_1_11 , f1_1_13  = Feature_Extractor(writer1_1)
    f1_2_5 , f1_2_11 , f1_2_13  = Feature_Extractor(writer1_2)
    f2_1_5 , f2_1_11, f2_1_13  = Feature_Extractor(writer2_1)
    f2_2_5, f2_2_11 , f2_2_13 = Feature_Extractor(writer2_2)
    f3_1_5 , f3_1_11 , f3_1_13  = Feature_Extractor(writer3_1)
    f3_2_5 , f3_2_11 , f3_2_13  = Feature_Extractor(writer3_2)
    pred_f5 , pred_f11 , pred_f13  = Feature_Extractor(test_img)
    '''
    
    f1_1  = Feature_Extractor(writer1_1)
    f1_2  = Feature_Extractor(writer1_2)
    f2_1  = Feature_Extractor(writer2_1)
    f2_2 = Feature_Extractor(writer2_2)
    f3_1  = Feature_Extractor(writer3_1)
    f3_2  = Feature_Extractor(writer3_2)
    pred  = Feature_Extractor(test_img)
    
    features = np.array([f1_1,f1_2,f2_1,f2_2,f3_1,f3_2])
    #features_f5 = np.array([f1_1_5,f1_2_5,f2_1_5,f2_2_5,f3_1_5,f3_2_5])
    #features_f11 = np.array([f1_1_11,f1_2_11,f2_1_11,f2_2_11,f3_1_11,f3_2_11])
  # features_f11_f13 = np.array([list(f1_1_11)+list(f1_1_13),list(f1_2_11)+list(f1_2_13),list(f2_1_11)+list(f2_1_13),list(f2_2_11)+list(f2_2_13),list(f3_1_11)+list(f3_1_13),list(f3_2_11)+list(f3_2_13)])
    
    
    classifier = SVC()
    classifier.fit(features, [1,1,2,2,3,3])
    result = classifier.predict([pred])
    
    '''classifier_f11 = SVC()
    classifier_f11.fit(features_f11, [1,1,2,2,3,3])
    result_f11 = classifier_f11.predict([pred_f11])
    
    classifier_f11_f13 = SVC()
    classifier_f11_f13.fit(features_f11_f13, [1,1,2,2,3,3])
    result_f11_f13 = classifier_f11_f13.predict([list(pred_f11)+list(pred_f13)])
    result = np.array([result_f5[0],result_f11[0],result_f11_f13[0]])
    vote_1 = len(result[result==1])
    vote_2 = len(result[result==2])
    vote_3 = len(result[result==3])
    if vote_1 >= 2 :
        result = 1
    elif vote_2 >= 2 :
        result = 2
    elif vote_3 >=2 :
        result = 3
    else:
        result = result_f5
   '''
    
    
    en = time.time()
    print("Time Taken : ",(en-st))
    print(result,correct[i])
    if(result==correct[i]):
        acc+=1
    i+=1
    print(i,acc, "Accuracy : ",acc*100/i,"%")

Time Taken :  6.674614906311035
[3] 3
1 1 Accuracy :  100.0 %
Time Taken :  8.368782758712769
[3] 3
2 2 Accuracy :  100.0 %
Time Taken :  7.93947172164917
[3] 3
3 3 Accuracy :  100.0 %
Time Taken :  7.170639276504517
[3] 3
4 4 Accuracy :  100.0 %
Time Taken :  6.828195095062256
[3] 3
5 5 Accuracy :  100.0 %
Time Taken :  8.787277936935425
[3] 3
6 6 Accuracy :  100.0 %
Time Taken :  8.544281959533691
[3] 3
7 7 Accuracy :  100.0 %
Time Taken :  6.417924880981445
[3] 3
8 8 Accuracy :  100.0 %
Time Taken :  5.316935300827026
[3] 3
9 9 Accuracy :  100.0 %
Time Taken :  6.831123352050781
[3] 3
10 10 Accuracy :  100.0 %
Time Taken :  7.958348035812378
[3] 3
11 11 Accuracy :  100.0 %
Time Taken :  7.122500419616699
[3] 3
12 12 Accuracy :  100.0 %
Time Taken :  7.974433660507202
[3] 3
13 13 Accuracy :  100.0 %
Time Taken :  7.956566095352173
[3] 3
14 14 Accuracy :  100.0 %
Time Taken :  7.795902729034424
[3] 3
15 15 Accuracy :  100.0 %
Time Taken :  7.3915019035339355
[3] 3
16 16 Accuracy :  10

# Results 
### SVM on F1 Feature : 95.7%  (68/71)
### SVM on F5 Feature : 97.2%  (69/71)
### SVM on F4 Feature : 97.2%  (69/71) With Resize /4  , 98.6% (70/71)  With Resize /2 
### SVM on F11 Feature : 94.3% (67/71)
### SVM on F11+F12+F13+F14   : 83.1% (59/71)
### SVM on  F11 + F13 : 93% (66/71)
###  3 SVM Classifiers   (F11+F13) , F11 , F5 : 95.7%  (68/71)

# Random Test


In [14]:
with open("forms.txt") as h:
    lines = h.readlines()[16:]

In [15]:
freq = np.zeros(680, dtype=int)
formWriter = {}
for i in lines:
    line = i.split(' ')
    key = line[1]
    if key in formWriter.keys():
        formWriter[key].append(line[0])
    else:
        formWriter[key] = [line[0]]

In [21]:
freq = [x for x, l in formWriter.items() if len(l)>2]

In [30]:
acc = 0
total = 0
for itr in range(500):
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
        forms = [freq[k]+'-'+'{:03d}'.format(s)+'.jpg' for s in range(len(forms))]
        #idxes = np.random.randint(len(forms), size=3)
        idxes = set()
        while len(idxes) !=3: #To avoid test image same as train images
            idxes.add(np.random.randint(len(forms), size=1)[0])
        idxes = list(idxes)
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    Y = []
    for i in range(len(train)):
        img = cv2.imread('train/'+train[i], cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
       
        f = Feature_Extractor(img)
        trainFeatures.append(f)
        Y.append(i//2)
    trainFeatures =  np.array(trainFeatures)
    classifier = SVC()
   
    classifier.fit(trainFeatures, Y)
    
    img = cv2.imread("train/"+testImg, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    testFeatures = Feature_Extractor(img)
    result = classifier.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
    total +=1
    print(len(testFeatures))
    print(total,acc, "Accuracy : ",acc*100/total,"%")
    print(result, testWriter)
    
    
    



512
1 1 Accuracy :  100.0 %
[1] 1
512
2 2 Accuracy :  100.0 %
[2] 2
512
3 3 Accuracy :  100.0 %
[2] 2
512
4 4 Accuracy :  100.0 %
[1] 1
512
5 5 Accuracy :  100.0 %
[1] 1
512
6 6 Accuracy :  100.0 %
[0] 0
512
7 7 Accuracy :  100.0 %
[2] 2
512
8 8 Accuracy :  100.0 %
[0] 0
512
9 9 Accuracy :  100.0 %
[1] 1
512
10 10 Accuracy :  100.0 %
[0] 0
512
11 11 Accuracy :  100.0 %
[1] 1
512
12 11 Accuracy :  91.66666666666667 %
[0] 1
512
13 12 Accuracy :  92.3076923076923 %
[1] 1
512
14 13 Accuracy :  92.85714285714286 %
[1] 1
512
15 14 Accuracy :  93.33333333333333 %
[0] 0
512
16 15 Accuracy :  93.75 %
[2] 2
512
17 16 Accuracy :  94.11764705882354 %
[2] 2
512
18 17 Accuracy :  94.44444444444444 %
[0] 0
512
19 18 Accuracy :  94.73684210526316 %
[0] 0
512
20 19 Accuracy :  95.0 %
[1] 1
512
21 20 Accuracy :  95.23809523809524 %
[1] 1
512
22 21 Accuracy :  95.45454545454545 %
[1] 1
512
23 22 Accuracy :  95.65217391304348 %
[1] 1
512
24 23 Accuracy :  95.83333333333333 %
[0] 0
512
25 24 Accuracy :  96

512
176 170 Accuracy :  96.5909090909091 %
[1] 1
512
177 171 Accuracy :  96.61016949152543 %
[1] 1
512
178 172 Accuracy :  96.62921348314607 %
[2] 2
512
179 173 Accuracy :  96.64804469273743 %
[1] 1
512
180 174 Accuracy :  96.66666666666667 %
[1] 1
512
181 175 Accuracy :  96.68508287292818 %
[1] 1
512
182 176 Accuracy :  96.7032967032967 %
[1] 1
512
183 177 Accuracy :  96.72131147540983 %
[1] 1
512
184 178 Accuracy :  96.73913043478261 %
[1] 1
512
185 179 Accuracy :  96.75675675675676 %
[1] 1
512
186 180 Accuracy :  96.7741935483871 %
[1] 1
512
187 181 Accuracy :  96.79144385026738 %
[1] 1
512
188 182 Accuracy :  96.80851063829788 %
[2] 2
512
189 183 Accuracy :  96.82539682539682 %
[2] 2
512
190 184 Accuracy :  96.84210526315789 %
[1] 1
512
191 185 Accuracy :  96.8586387434555 %
[0] 0
512
192 186 Accuracy :  96.875 %
[0] 0
512
193 187 Accuracy :  96.89119170984456 %
[0] 0
512
194 188 Accuracy :  96.90721649484536 %
[0] 0
512
195 188 Accuracy :  96.41025641025641 %
[2] 0
512
196 189 Acc

512
343 330 Accuracy :  96.20991253644316 %
[2] 2
512
344 331 Accuracy :  96.22093023255815 %
[1] 1
512
345 332 Accuracy :  96.23188405797102 %
[1] 1
512
346 333 Accuracy :  96.24277456647398 %
[2] 2
512
347 334 Accuracy :  96.25360230547551 %
[1] 1
512
348 335 Accuracy :  96.26436781609195 %
[1] 1
512
349 336 Accuracy :  96.27507163323783 %
[1] 1
512
350 337 Accuracy :  96.28571428571429 %
[2] 2
512
351 338 Accuracy :  96.29629629629629 %
[0] 0
512
352 339 Accuracy :  96.30681818181819 %
[1] 1
512
353 340 Accuracy :  96.3172804532578 %
[1] 1
512
354 341 Accuracy :  96.32768361581921 %
[0] 0
512
355 342 Accuracy :  96.33802816901408 %
[2] 2
512
356 343 Accuracy :  96.34831460674157 %
[2] 2
512
357 344 Accuracy :  96.35854341736695 %
[1] 1
512
358 345 Accuracy :  96.36871508379889 %
[0] 0
512
359 346 Accuracy :  96.37883008356546 %
[1] 1
512
360 347 Accuracy :  96.38888888888889 %
[0] 0
512
361 348 Accuracy :  96.39889196675901 %
[1] 1
512
362 349 Accuracy :  96.40883977900552 %
[1] 1
5

# Results 
### SVM :  Normal Windows , No Ali part  : 86%    (86/100)     , 83%
### KNN :  Normal Windows , No Ali part  : 74%    (74/100)
### SVM :  Normal Windows , No Ali part no local features (84/100)
### SVM :  Normal Windwos , No Ali part , Pca  :64% (7/11)
### SVM : F1  90% (449/500)
### SVM : F5  97.2% (484/498)
### SVM : F11 90.6% (453/500)
### SVM : F4  95.2%  (474/498)
### KNN : F5 93.4%  (467/500)